<a href="https://colab.research.google.com/github/ort-eila/git_kundaje_annotations/blob/main/step_2_load_dataset_in_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 2 - load dataset

In [ ]:
# !pip install transformers datasets faiss-cpu psutil

In [ ]:
import torch
from datasets import Features, Sequence, Value, load_dataset
import os
from functools import partial
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import List, Optional

from datasets import Features, Sequence, Value, load_dataset
import faiss
from transformers import (
    DPRContextEncoder,
    DPRContextEncoderTokenizerFast,
    RagRetriever,
    RagSequenceForGeneration,
    RagTokenizer,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls drive/MyDrive/bio-llm/wikipedia_details.tsv

drive/MyDrive/bio-llm/wikipedia_details.tsv


In [ ]:
!ls drive/MyDrive/bio-llm/output

my_knowledge_dataset		       wiki_dataset
my_knowledge_dataset_hnsw_index.faiss  wiki_index.faiss


In [ ]:
CSV_FILE = 'wikipedia_details.tsv'
# Define the destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/bio-llm/'
BATCH_SIZE = 16
NUM_PROCESSES = 2
WIKI_DATASET_NAME = "wiki_dataset"
N = 200 # this is the size of the split
MAX_LENGTH = 60

# Either 'facebook/rag-sequence-nq' or 'facebook/rag-token-nq'"},
RAG_MODEL_NAME = "facebook/rag-sequence-nq"
DPR_CTX_ENCODER = "facebook/dpr-ctx_encoder-multiset-base"

OUTPUT_DIR = os.path.join(destination_folder,"output_notebook")
CSV_PATH = os.path.join(destination_folder,CSV_FILE)

print("CSV_PATH is ",CSV_PATH)

HNSW_FAISS_IDX_DIM = 768
HNSW_BI_DIR_LINKS_CREATED_FOR_IDX = 128

CSV_PATH is  /content/drive/MyDrive/bio-llm/wikipedia_details.tsv


In [ ]:
!rm -rf $OUTPUT_DIR
!mkdir $OUTPUT_DIR

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


def split_text(text: str, n=N, character=" ") -> List[str]:
    """Split the text every ``n``-th occurrence of ``character``"""
    text = text.split(character)
    print("split_text. text is {}".format(text))
    return [character.join(text[i : i + n]).strip() for i in range(0, len(text), n)]

def split_documents(documents: dict) -> dict:
    """Split documents into passages"""
    titles, texts = [], []
    cnt = 1
    for title, text in zip(documents["title"], documents["text"]):
      #  if title == "title":
      #   print("****title is {}. continue".format(title))
      #    # continue
      #   print("!!!title is {} and text is {}\n".format(title,text))
        if text is not None:
            print("title is {}. text len is {} text is {}\n".format(title,len(text),text))
            split_text_output = split_text(text)
            print("len is {}. type is {}. split_text_output is {}. ".format(len(split_text_output) ,type(split_text_output),split_text_output))
            print("last item is split_text_output[-1]: {}".format(split_text_output[-1]))
            for passage in split_text(text):
                print("!!!!For title {} passage is {} ".format(title, passage))
                titles.append(title if title is not None else "")
                texts.append(passage)
        else:
          print("text is None\n")
    print("split_documents returns title: {} and text {}\n".format(title, text))
    return {"title": titles, "text": texts}


# model(input_ids).pooler_output

def embed(documents: dict, ctx_encoder: DPRContextEncoder, ctx_tokenizer: DPRContextEncoderTokenizerFast) -> dict:
    """Compute the DPR embeddings of document passages"""
    input_ids = ctx_tokenizer(
        documents["title"], documents["text"], truncation=True, padding="longest", return_tensors="pt"
    )["input_ids"]
    embeddings = ctx_encoder(input_ids.to(device=device), return_dict=True).pooler_output
    return {"embeddings": embeddings.detach().cpu().numpy()}


In [ ]:
def print_dataset(dataset,idx,end_idx=len(dataset)):
    if len(dataset) > 0:
        for i in range(idx, end_idx):  # Print the first 5 rows or less if dataset is smaller
            print(f"Title: {dataset['title'][i]}")
            print(f"Text: {dataset['text'][i]}")
            print(f"embeddings: {dataset['embeddings'][i]}")
            print()
    else:
        print("The dataset is empty.")

In [ ]:
# DPR, which stands for Dense Passage Retrieval, is a method for representing passages of text in a
# dense vector space, such that similar passages are close to each other in this space.
# DPR is commonly used in information retrieval and question-answering systems to efficiently
# retrieve relevant passages given a query.

In [ ]:
######################################
print("Step 1 - Create the dataset")
######################################

# The dataset needed for RAG must have three columns:
# - title (string): title of the document
# - text (string): text of a passage of the document
# - embeddings (array of dimension d): DPR representation of the passage

print("Step 1 - 1 split docs")
# You can load a Dataset object this way
dataset = load_dataset(
    "csv", data_files=[CSV_PATH], split="train", delimiter="\t", column_names=["title", "text"]
)

# More info about loading csv files in the documentation: https://huggingface.co/docs/datasets/loading_datasets.html?highlight=csv#csv-files

# Then split the documents into passages.
dataset = dataset.map(split_documents, batched=True, num_proc=NUM_PROCESSES)

# up to here, we are splitting to the sentences in the datasets that are going to be enbedded into numerical vectors

Step 1 - Create the dataset


Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

title is title. text len is 4 text is text

split_text. text is ['text']
len is 1. type is <class 'list'>. split_text_output is ['text']. 
title is BRCA_mutation. text len is 35932 text is A BRCA mutation is a mutation in either of the BRCA1 and BRCA2 genes, which are tumour suppressor genes. Hundreds of different types of mutations in these genes have been identified, some of which have been determined to be harmful, while others have no proven impact. Harmful mutations in these genes may produce a hereditary breast–ovarian cancer syndrome in affected persons. Only 5–10% of breast cancer cases in women are attributed to BRCA1 and BRCA2 mutations (with BRCA1 mutations being slightly more common than BRCA2 mutations), but the impact on women with the gene mutation is more profound.[2] Women with harmful mutations in either BRCA1 or BRCA2 have a risk of breast cancer that is about five times the normal risk, and a risk of ovarian cancer that is about ten to thirty times normal.[3] The ri

In [ ]:
# used encoder to for the context of the dataset.
# adding the embeddign with the encoding inormation
print("Step 1 - 2 add embedding based on the DPR_CTX_ENCODER. ctx stands for context")
print("DPR_CTX_ENCODER is {}".format(DPR_CTX_ENCODER))
print("BATCH_SIZE is ",BATCH_SIZE)
# And compute the embeddings
ctx_encoder = DPRContextEncoder.from_pretrained(DPR_CTX_ENCODER).to(device=device)
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained(DPR_CTX_ENCODER)
new_features = Features(
    {"text": Value("string"), "title": Value("string"), "embeddings": Sequence(Value("float32"))}
)  # optional, save as float32 instead of float64 to save space
dataset = dataset.map(
    partial(embed, ctx_encoder=ctx_encoder, ctx_tokenizer=ctx_tokenizer),
    batched=True,
    batch_size=BATCH_SIZE,
    features=new_features,
)
print("dataset.shape is ",dataset.shape)

# save the dataset

passages_path = os.path.join(OUTPUT_DIR, "{}".format(WIKI_DATASET_NAME))
print("passages_path is {}".format(passages_path))
dataset.save_to_disk(passages_path)
# from datasets import load_from_disk
# dataset = load_from_disk(passages_path)  # to reload the dataset

DPR_CTX_ENCODER is facebook/dpr-ctx_encoder-multiset-base
BATCH_SIZE is  16


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenize

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

(29, 3)

In [ ]:
print_dataset(dataset,5,7)

Title: BRCA_mutation
Text: average because it is unusually susceptible to platinum-based chemotherapy like cisplatin.[9]: 275–302  BRCA1-related ovarian cancer appears at younger ages, but the risk for women with BRCA2 climbs markedly at or shortly after menopause.[9]: 275–302 A 25-year-old woman with no mutation in her BRCA genes has an 84% probability to reach at least the age of 70.[14] Of those not surviving, 11% die from either breast or ovarian cancer, and 89% from other causes.Compared to that, a woman with a high-risk BRCA1 mutation, if she had breast cancer screening but no prophylactic medical or surgical intervention, would have only 59% chance to reach age 70, twenty-five percentage points lower than normal. Of those women not surviving, 26% would die of breast cancer, 46% ovarian cancer, and 28% other causes.[14]Women with high-risk BRCA2 mutations, with screening but with no prophylactic medical or surgical intervention, would have only 71% chance to reach age 70, thirtee

In [ ]:
print_dataset(dataset,27)

Title: BRCA_mutation
Text: Utah, National Institute of Environmental Health Sciences (NIEHS) and Myriad Genetics in 1994;[5] over the next year, Myriad, in collaboration with investigators from Endo Recherche, Inc., HSC Research & Development Limited Partnership, and University of Pennsylvania, isolated and sequenced the BRCA2 gene and identified key mutations, and the first BRCA2 patent was filed in the US by Myriad and other institutions in 1995.[6] Myriad is the exclusive licensee of these patents and has enforced them in the US against clinical diagnostic labs.[8] This business model led to Myriad growing being a startup in 1994 to being a publicly traded company with 1200 employees and about $500M in annual revenue in 2012;[7] it also led to controversy over high prices and the inability to get second opinions from other diagnostic labs, which in turn led to the landmark Association for Molecular Pathology v. Myriad Genetics lawsuit.[8][49] The patents began to expire in 2014.Acco

In [ ]:
# Next: index the dataset dataset.
######################################
print("Step 2 - Index the dataset with faiss library")
######################################

print("HNSW_FAISS_IDX_DIM is {} and HNSW_BI_DIR_LINKS_CREATED_FOR_IDX is {}".format(HNSW_FAISS_IDX_DIM,HNSW_BI_DIR_LINKS_CREATED_FOR_IDX))

# Let's use the Faiss implementation of HNSW for fast approximate nearest neighbor search
index = faiss.IndexHNSWFlat(HNSW_FAISS_IDX_DIM, HNSW_BI_DIR_LINKS_CREATED_FOR_IDX, faiss.METRIC_INNER_PRODUCT)
dataset.add_faiss_index("embeddings", custom_index=index)

# And save the index
print("OUTPUT_DIR is ",OUTPUT_DIR)
index_path = os.path.join(OUTPUT_DIR, "{}.faiss".format(WIKI_DATASET_NAME))
print("index_path is ",index_path)

dataset.get_index("embeddings").save(index_path)
# dataset.load_faiss_index("embeddings", index_path)  # to reload the index

Step 2 - Index the dataset with faiss
HNSW_FAISS_IDX_DIM is 768 and HNSW_BI_DIR_LINKS_CREATED_FOR_IDX is 128


  0%|          | 0/1 [00:00<?, ?it/s]

OUTPUT_DIR is  /content/drive/MyDrive/bio-llm/output_notebook
index_path is  /content/drive/MyDrive/bio-llm/output_notebook/wiki_dataset.faiss


In [ ]:
######################################
print("Step 3 - Load RAG")
######################################

print("RAG_MODEL_NAME is ",RAG_MODEL_NAME)
# Easy way to load the model
retriever = RagRetriever.from_pretrained(
    RAG_MODEL_NAME, index_name="custom", indexed_dataset=dataset
)
model = RagSequenceForGeneration.from_pretrained(RAG_MODEL_NAME, retriever=retriever)
# print("model is ",model)
# RagTokenizer is an input
tokenizer = RagTokenizer.from_pretrained(RAG_MODEL_NAME)
print("RAG_MODEL tokenizer is ",tokenizer)

# For distributed fine-tuning you'll need to provide the paths instead, as the dataset and the index are loaded separately.
# retriever = RagRetriever.from_pretrained(rag_model_name, index_name="custom", passages_path=passages_path, index_path=index_path)


Step 3 - Load RAG
RAG_MODEL_NAME is  facebook/rag-sequence-nq


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function 

RAG_MODEL tokenizer is  <transformers.models.rag.tokenization_rag.RagTokenizer object at 0x7fbf73872620>


In [ ]:
####### step 4: RAG model execution. TODO: play with similarity ##########


import torch
from scipy.spatial.distance import cosine
from transformers import RagRetriever, RagSequenceForGeneration, RagTokenizer, pipeline


# Ask a question
question = "what is BRCA mutations?"
print("question is {}".format(question))
# use the RAG Model tokenizer
input_ids = tokenizer(question, return_tensors="pt")["input_ids"]
# generate the output
generated = model.generate(input_ids, max_length=MAX_LENGTH, num_return_sequences=1, num_beams=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, early_stopping=True)

# Decode the generated answer
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
print("Q: " + question)
print("A: " + generated_string)



question is what is BRCA mutations?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Q: what is BRCA mutations?
A:  brca1


In [ ]:
# question = rag_args.question #or "What does Moses' rod turn into ?"
question = "what is residue ?"
print("question is {}".format(question))
input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]
generated = model.generate(input_ids,max_length=MAX_LENGTH)
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
logger.info("Q: " + question)
logger.info("A: " + generated_string)
print("answer is ",generated_string)

question is what is residue ?
answer is   amino acid


In [ ]:
generated_string

' amino acid'

In [ ]:
(input_ids).squeeze(0)

tensor([  101,  2054,  2003, 21755,  1029,   102])

In [ ]:
# model.get_input_embeddings((input_ids).squeeze(0))